<a href="https://colab.research.google.com/github/r2barati/TREC23-CrisisFACTS/blob/main/Pre_trained_BERT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade git+https://github.com/allenai/ir_datasets.git@crisisfacts # install ir_datasets (crisisfacts branch)


In [ ]:
# Set up credentials
credentials = {
    "institution": "<Toronto Metropolitan University>", # University, Company or Public Agency Name
    "contactname": "<Reza Barati, Aary Kartha>", # Your Name
    "email": "<rezabarati@gmail.com, aaryaman.kartha@torontomu.ca>", # A contact email address
    "institutiontype": "<Research>" # Either 'Research', 'Industry', or 'Public Sector'
}

  # Write this to a file so it can be read when needed
import json
import os

home_dir = os.path.expanduser('~')

!mkdir -p ~/.ir_datasets/auth/
with open(home_dir + '/.ir_datasets/auth/crisisfacts.json', 'w') as f:
    json.dump(credentials, f)


In [ ]:
# Define the event numbers
eventNoList = [
    "001", # Lilac Wildfire 2017
]

In [ ]:
# Define the function to get days for an event
import requests

# Gets the list of days for a specified event number, e.g. '001'
def getDaysForEventNo(eventNo):

    # We will download a file containing the day list for an event
    url = "http://trecis.org/CrisisFACTs/CrisisFACTS-"+eventNo+".requests.json"

    # Download the list and parse as JSON
    dayList = requests.get(url).json()

    # Print each day
    # Note each day object contains the following fields
    #   {
    #      "eventID" : "CrisisFACTS-001",
    #      "requestID" : "CrisisFACTS-001-r3",
    #      "dateString" : "2017-12-07",
    #      "startUnixTimestamp" : 1512604800,
    #      "endUnixTimestamp" : 1512691199
    #   }

    return dayList

In [ ]:
# Use the function to get days for the first event
# Below has a manual function for getting the days of each event number (there are 0-17 )
import pandas as pd
for day in getDaysForEventNo(eventNoList[0]):
      print(day["dateString"])
    # print(day) day include Summary Request JSON File


2017-12-07
2017-12-08
2017-12-09
2017-12-10
2017-12-11
2017-12-12
2017-12-13
2017-12-14
2017-12-15


In [ ]:
# Get days for all events
eventsMeta = {}

for eventNo in eventNoList: # for each event
    dailyInfo = getDaysForEventNo(eventNo) # get the list of days
    eventsMeta[eventNo]= dailyInfo

    print("Event "+eventNo)
    for day in dailyInfo: # for each day
        print("  crisisfacts/"+eventNo+"/"+day["dateString"], "-->", day["requestID"]) # construct the request string

    print()

Event 001
  crisisfacts/001/2017-12-07 --> CrisisFACTS-001-r3
  crisisfacts/001/2017-12-08 --> CrisisFACTS-001-r4
  crisisfacts/001/2017-12-09 --> CrisisFACTS-001-r5
  crisisfacts/001/2017-12-10 --> CrisisFACTS-001-r6
  crisisfacts/001/2017-12-11 --> CrisisFACTS-001-r7
  crisisfacts/001/2017-12-12 --> CrisisFACTS-001-r8
  crisisfacts/001/2017-12-13 --> CrisisFACTS-001-r9
  crisisfacts/001/2017-12-14 --> CrisisFACTS-001-r10
  crisisfacts/001/2017-12-15 --> CrisisFACTS-001-r11



In [ ]:
# Download and print data for the second day of the ninth event
import ir_datasets

dataset = ir_datasets.load('crisisfacts/001/2017-12-07')

for item in dataset.docs_iter()[:1]:
    print(item)

In [ ]:
# Convert the stream of items to a Pandas DataFrame and filter by source type
import pandas as pd

# Convert the stream of items to a Pandas Dataframe
itemsAsDataFrame = pd.DataFrame(dataset.docs_iter())

# Create a copy of the first 100 rows
itemsAsDataFrame10000 = itemsAsDataFrame.head(10000).copy()
itemsAsDataFrame10000.to_csv("2017-12-07-10000.csv", index=False)

# Create a filter expression
is_reddit =  itemsAsDataFrame['source_type']=="Reddit"

# Apply our filter
itemsAsDataFrame[is_reddit]
itemsAsDataFrame10000_Reddit = itemsAsDataFrame[is_reddit].head(10000).copy()

# Create a filter expression
is_twitter =  itemsAsDataFrame['source_type']=="Twitter"

# Apply our filter
itemsAsDataFrame[is_twitter]
itemsAsDataFrame10000_Twitter = itemsAsDataFrame[is_twitter].head(10000).copy()

# Create a filter expression
is_fb =  itemsAsDataFrame['source_type']=="Facebook"

# Apply our filter
itemsAsDataFrame[is_fb]
itemsAsDataFrame10000_fb = itemsAsDataFrame[is_fb].head(10000).copy()

# Create a filter expression
is_news =  itemsAsDataFrame['source_type']=="News"

# Apply our filter
itemsAsDataFrame[is_news]
itemsAsDataFrame10000_news = itemsAsDataFrame[is_news].head(10000).copy()

itemsAsDataFrame.head(5)


In [ ]:
from google.colab import data_table

In [ ]:
User_Profiles_Event_Definition = pd.DataFrame(dataset.queries_iter())

In [ ]:
import pandas as pd

# documents_df = pd.read_csv('2017-12-07-10000.csv.csv')

In [ ]:
itemsAsDataFrame.head()

In [ ]:
User_Profiles_Event_Definition.head()

In [ ]:
!pip install sentence-transformers

In [ ]:
pip install tqdm

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertForMaskedLM, BertTokenizer
from tqdm import tqdm
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

# Define the base save path in Google Drive
save_base_path = "/content/gdrive/MyDrive/colab/crisisfacts/model"
os.makedirs(save_base_path, exist_ok=True)

# Function to get the last saved checkpoint path
def get_last_checkpoint(base_path):
    checkpoints = [os.path.join(base_path, name) for name in os.listdir(base_path)]
    return max(checkpoints, key=os.path.getctime) if checkpoints else None

# Check if there's a last saved checkpoint
last_checkpoint = get_last_checkpoint(save_base_path)

# Load BERT and tokenizer
if last_checkpoint:
    model = BertForMaskedLM.from_pretrained(last_checkpoint)
    print(f"Model loaded from {last_checkpoint}")
else:
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare data (replace with your actual data preparation)
text_data = User_Profiles_Event_Definition['text'].tolist() + itemsAsDataFrame['text'].tolist()
encoded_data = tokenizer(text_data, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

# Masking - 15% of tokens
probability = 0.15
masked_input_ids = encoded_data['input_ids'].clone()
for i in range(masked_input_ids.size(0)):
    for j in range(masked_input_ids.size(1)):
        if torch.rand(1).item() < probability:
            masked_input_ids[i, j] = tokenizer.mask_token_id

# DataLoader
train_dataset = TensorDataset(masked_input_ids, encoded_data['attention_mask'], encoded_data['input_ids'])
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Calculate the number of batches that constitute 1% of the training
one_percent_batches = len(train_dataloader) // 100

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
num_epochs = 3
for epoch in range(num_epochs):
    for batch_num, batch in tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch+1}/{num_epochs}", total=len(train_dataloader)):
        inputs, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        # Save the model every 1% progress
        if (batch_num + 1) % one_percent_batches == 0:
            save_path = f"{save_base_path}/epoch{epoch+1}_batch{batch_num+1}"
            model.save_pretrained(save_path)
            print(f"Model saved at {save_path}")

    print(f"Epoch {epoch+1} completed.")

# Save the final model and tokenizer
final_save_path = save_base_path
model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)
print(f"Final model saved at {final_save_path}")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch 1/3:   1%|          | 4/459 [00:17<34:24,  4.54s/it]

Model saved at /content/gdrive/MyDrive/colab/crisisfacts/model/epoch1_batch4


Epoch 1/3:   2%|▏         | 8/459 [00:32<31:22,  4.18s/it]

...

Model saved at /content/gdrive/MyDrive/colab/crisisfacts/model/epoch3_batch452


Epoch 3/3:  99%|█████████▉| 456/459 [15:24<00:06,  2.12s/it]

Model saved at /content/gdrive/MyDrive/colab/crisisfacts/model/epoch3_batch456


Epoch 3/3: 100%|██████████| 459/459 [15:30<00:00,  2.03s/it]


Epoch 3 completed.
Final model saved at /content/gdrive/MyDrive/colab/crisisfacts/model


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/MyDrive/colab/crisisfacts/model"


In [ ]:
!pip install transformers
from transformers import BertForMaskedLM, BertTokenizer

save_base_path = "/content/gdrive/MyDrive/colab/crisisfacts/model"

model = BertForMaskedLM.from_pretrained(save_base_path)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00


In [ ]:
def predict_masked_text(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding='max_length', max_length=128)

    # Predict the masked tokens
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)

    # Decode the predictions to text
    predicted_text = tokenizer.decode(predictions[0])

    return predicted_text


In [ ]:
def clean_predicted_text(predicted_tokens):
    # Remove special tokens
    readable_text = predicted_tokens.replace("[CLS]", "").replace("[SEP]", "")

    # Remove padding tokens
    readable_text = readable_text.replace("[PAD]", "").strip()

    return readable_text

In [ ]:
predicted_text = "[CLS] the hurricane caused moderate damage. [SEP] [PAD] [PAD] ..."
cleaned_text = clean_predicted_text(predicted_text)
print(cleaned_text)  # Output: "the hurricane caused moderate damage."


the hurricane caused moderate damage.    ...


In [ ]:
import torch

result = predict_masked_text("The hurricane caused [MASK] damage.")
print(result)


[CLS] the hurricane caused moderate damage. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [ ]:
from tqdm.notebook import tqdm
import time
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to create embeddings using the trained BERT model
def create_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model.bert(**inputs) # Use the underlying BERT model
    return outputs.last_hidden_state.mean(dim=1) # Average the token embeddings to get a single vector

start_time = time.time()

# Create embeddings for queries
query_embeddings = [create_embedding(User_Profiles_Event_Definition.iloc[idx]['text']).numpy() for idx in tqdm(range(len(User_Profiles_Event_Definition)), desc="Processing Queries")]

# Create embeddings for documents
document_embeddings = []
for eventNo in tqdm(eventNoList, desc="Processing Documents"):
    dataset = ir_datasets.load(f'crisisfacts/{eventNo}/2017-12-07') # Adjust the date as needed
    for item in dataset.docs_iter():
        document_embeddings.append(create_embedding(item.text).numpy())

# Compute similarity matrix
similarity_matrix = cosine_similarity(np.vstack(query_embeddings), np.vstack(document_embeddings))

# Get top 5 indices of most similar documents for each query
top_indices = np.argsort(-similarity_matrix, axis=1)[:, :5]

# Retrieve and print the corresponding documents for each query
document_list = [item.text for eventNo in eventNoList for item in ir_datasets.load(f'crisisfacts/{eventNo}/2017-12-07').docs_iter()]
for query_idx, indices in enumerate(top_indices):
    query_text = User_Profiles_Event_Definition.iloc[query_idx]['text']
    print(f"Query: {query_text}")
    print("Top 5 Relevant Documents:")
    for idx in indices:
        document_text = document_list[idx]
        print(f"  - {document_text}")
    print("\n" + "="*50 + "\n")

end_time = time.time()
print(f"Total execution time: {end_time - start_time:.2f} seconds")


Processing Queries:   0%|          | 0/52 [00:00<?, ?it/s]

Processing Documents:   0%|          | 0/1 [00:00<?, ?it/s]

Query: Have airports closed
Top 5 Relevant Documents:
  - OTHER CLOSURES  Legoland California will be closed Friday out of consideration for employees affected by the recent fires.
  - ROAD AND FREEWAY CLOSURES  All roadways that were previously closed are now open.
  - Officials have lifted both evacuations and road closures caused by the fire.
  - #Breaking #LilacFire 150 acres. update: 2 structures have burned down, 12 damaged.  no word if these are homes or not.
  - “There are horses running all loose down here,” Sise told the racing publication.


Query: Have railways closed
Top 5 Relevant Documents:
  - “There are horses running all loose down here,” Sise told the racing publication.
  - OTHER CLOSURES  Legoland California will be closed Friday out of consideration for employees affected by the recent fires.
  - 519 power poles have been damaged by the Thomas fire.
  - ROAD AND FREEWAY CLOSURES  All roadways that were previously closed are now open.
  - @BDeniSE home will always 

## Model Overview

### Data Retrieval and Preprocessing
Our model begins by defining specific events and their associated disaster-day pairs. Through a series of functions and API calls, we retrieve and preprocess the required data, converting it into a format amenable to analysis.

### Fine-Tuning BERT for Masked Language Modeling (MLM)
We utilize the BERT model, fine-tuned using unsupervised learning with the MLM task. The fine-tuning process involves masking specific tokens in the input text and training the model to predict the masked words. This helps the model to understand the semantics and structure of the language, especially in the context of disaster-related content.

## Potential Issues

1. **Lack of Fact Extraction Mechanism**: The current model focuses on MLM fine-tuning but lacks a dedicated mechanism for extracting atomic facts and importance scores as required by the task.
2. **Redundancy Management**: The task requires a minimally redundant list of facts, and the current model does not have a clear method to handle redundancy.
3. **Contextual Understanding**: Although BERT is a powerful model, it may not fully capture the specific nuances and contextual needs of disaster-related information.

## Future Improvements and Fine-Tuning

1. **Integrate Fact Extraction Layer**: We plan to integrate a specific layer for fact extraction, capable of producing atomic facts and associating importance scores with them.
2. **Implement Redundancy Reduction**: A mechanism will be developed to ensure minimal redundancy in the extracted facts.
3. **Utilize Domain-Specific Pretraining**: By further fine-tuning the model on domain-specific data, we can enhance its understanding of disaster-related terminology and context.

In [ ]:
# Testing Against 2023 Data
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Assuming `text_data` is your combined text data
text_data = User_Profiles_Event_Definition['text'].tolist() + itemsAsDataFrame['text'].tolist()

# Split into training and test data
train_data, test_data = train_test_split(text_data, test_size=0.2, random_state=42)

# Preprocess training data
train_encoded_data = tokenizer(train_data, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

# Preprocess test data
test_encoded_data = tokenizer(test_data, truncation=True, padding='max_length', max_length=128, return_tensors='pt')

# Masking for test data
test_masked_input_ids = test_encoded_data['input_ids'].clone()
for i in range(test_masked_input_ids.size(0)):
    for j in range(test_masked_input_ids.size(1)):
        if torch.rand(1).item() < probability:
            test_masked_input_ids[i, j] = tokenizer.mask_token_id

# DataLoader for test data
test_dataset = TensorDataset(test_masked_input_ids, test_encoded_data['attention_mask'], test_encoded_data['input_ids'])
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Evaluate on test data
model.eval()
total_loss = 0
for batch in test_dataloader:
    inputs, attention_mask, labels = batch
    with torch.no_grad():
        outputs = model(inputs, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
average_test_loss = total_loss / len(test_dataloader)
print(f"Test Loss: {average_test_loss}")


NameError: ignored